b and c

In [11]:
library(car)
library(lmtest)

In [12]:
library(haven)
hprice1 <- read_dta("data_ps2/hprice1.dta")

In [13]:
colnames(hprice1)

[1] "price"    "assess"   "bdrms"    "lotsize"  "sqrft"    "colonial"
 [7] "lprice"   "lassess"  "llotsize" "lsqrft"

In [14]:
summary(hprice1)

     price           assess          bdrms          lotsize          sqrft     
 Min.   :111.0   Min.   :198.7   Min.   :2.000   Min.   : 1000   Min.   :1171  
 1st Qu.:230.0   1st Qu.:253.9   1st Qu.:3.000   1st Qu.: 5733   1st Qu.:1660  
 Median :265.5   Median :290.2   Median :3.000   Median : 6430   Median :1845  
 Mean   :293.5   Mean   :315.7   Mean   :3.568   Mean   : 9020   Mean   :2014  
 3rd Qu.:326.2   3rd Qu.:352.1   3rd Qu.:4.000   3rd Qu.: 8583   3rd Qu.:2227  
 Max.   :725.0   Max.   :708.6   Max.   :7.000   Max.   :92681   Max.   :3880  
    colonial          lprice         lassess         llotsize     
 Min.   :0.0000   Min.   :4.710   Min.   :5.292   Min.   : 6.908  
 1st Qu.:0.0000   1st Qu.:5.438   1st Qu.:5.537   1st Qu.: 8.654  
 Median :1.0000   Median :5.582   Median :5.671   Median : 8.769  
 Mean   :0.6932   Mean   :5.633   Mean   :5.718   Mean   : 8.905  
 3rd Qu.:1.0000   3rd Qu.:5.788   3rd Qu.:5.864   3rd Qu.: 9.058  
 Max.   :1.0000   Max.   :6.586   Max.

In [15]:
lprice <- log(hprice1$price)
llotsize <- log(hprice1$lotsize)
lsqrft <- log(hprice1$sqrft)

In [16]:
m <- lm(lprice ~ llotsize + lsqrft + bdrms, data = hprice1)
summary(m)


Call:
lm(formula = lprice ~ llotsize + lsqrft + bdrms, data = hprice1)

Residuals:
     Min       1Q   Median       3Q      Max 
-0.68422 -0.09178 -0.01584  0.11213  0.66899 

Coefficients:
            Estimate Std. Error t value Pr(>|t|)    
(Intercept) -1.29704    0.65128  -1.992   0.0497 *  
llotsize     0.16797    0.03828   4.388 3.31e-05 ***
lsqrft       0.70023    0.09287   7.540 5.01e-11 ***
bdrms        0.03696    0.02753   1.342   0.1831    
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1

Residual standard error: 0.1846 on 84 degrees of freedom
Multiple R-squared:  0.643,	Adjusted R-squared:  0.6302 
F-statistic: 50.42 on 3 and 84 DF,  p-value: < 2.2e-16


In [17]:
# regression with heteroscedasticity robust standard errors:
coeftest(m, vcov=hccm(m, type="hc1"))


t test of coefficients:

             Estimate Std. Error t value  Pr(>|t|)    
(Intercept) -1.297042   0.781314 -1.6601 0.1006279    
llotsize     0.167967   0.041473  4.0500 0.0001136 ***
lsqrft       0.700232   0.103829  6.7441 1.835e-09 ***
bdrms        0.036958   0.030601  1.2077 0.2305343    
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1


the robust standard errors are larger than the usual ones (except for the intercept), indicating the presence of heteroscedasticity in the model. But no change in the level of significance of the coefficients, meaning that the heteroscedasticity is not too severe.

Null hypothesis: errors are homoskedastic (constant variance).
Alternative: errors are heteroskedastic (variance depends on X).

In [21]:
# White test for heteroscedasticity

# get the residuals from the original regression
hprice1$resid <- m$residuals

# create the squared residuals
hprice1$residsq <- hprice1$resid^2

# create the additional variables needed for the White test
hprice1$llotsizesq <- hprice1$llotsize^2
hprice1$lsqrftsq <- hprice1$lsqrft^2
hprice1$bdrmssq <- hprice1$bdrms^2
hprice1$llotsizelsqrft <- hprice1$llotsize*hprice1$lsqrft
hprice1$llotsizebdrms <- hprice1$llotsize*hprice1$bdrms
hprice1$lsqrftbdrms <- hprice1$lsqrft*hprice1$bdrms

# run the auxiliary regression
reg_White <- lm(residsq ~ llotsize + lsqrft + bdrms + llotsizesq + lsqrftsq + bdrmssq + llotsizelsqrft + llotsizebdrms + lsqrftbdrms, data=hprice1)
output <- summary(reg_White)
output

# The F-statistic is:
F_stat <- output$fstatistic[1]

# The p-value is:
p_value <- pf(F_stat,9,78,lower.tail = F)
p_value

p_value < 0.05


Call:
lm(formula = residsq ~ llotsize + lsqrft + bdrms + llotsizesq + 
    lsqrftsq + bdrmssq + llotsizelsqrft + llotsizebdrms + lsqrftbdrms, 
    data = hprice1)

Residuals:
     Min       1Q   Median       3Q      Max 
-0.08636 -0.02717 -0.01197  0.00873  0.39929 

Coefficients:
                Estimate Std. Error t value Pr(>|t|)  
(Intercept)    12.206974   6.692663   1.824   0.0720 .
llotsize       -1.272828   0.708255  -1.797   0.0762 .
lsqrft         -1.756785   1.664948  -1.055   0.2946  
bdrms           0.287872   0.284522   1.012   0.3148  
llotsizesq      0.023520   0.016296   1.443   0.1529  
lsqrftsq        0.040274   0.123073   0.327   0.7444  
bdrmssq        -0.005091   0.009056  -0.562   0.5756  
llotsizelsqrft  0.120860   0.072130   1.676   0.0978 .
llotsizebdrms  -0.025276   0.032063  -0.788   0.4329  
lsqrftbdrms    -0.001095   0.048247  -0.023   0.9819  
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1

Residual standard error: 0.0734 on 78 degree

value 
0.4053127

value 
FALSE


- If *p* < α: reject H₀ → evidence of **heteroskedasticity**.  
- If *p* ≥ α: fail to reject H₀ → no evidence of heteroskedasticity.  


no evidence of heteroskedasticity (p-value = 0.4 > 0.05).

In [22]:
ceosal2 <- read_dta("data_ps2/ceosal2.dta")


In [23]:
reg1<-lm(lsalary ~ lsales + lmktval + profmarg + ceoten + comten, data=ceosal2)
summary(reg1)


Call:
lm(formula = lsalary ~ lsales + lmktval + profmarg + ceoten + 
    comten, data = ceosal2)

Residuals:
    Min      1Q  Median      3Q     Max 
-2.5436 -0.2796 -0.0164  0.2857  1.9879 

Coefficients:
             Estimate Std. Error t value Pr(>|t|)    
(Intercept)  4.571977   0.253466  18.038  < 2e-16 ***
lsales       0.187787   0.040003   4.694 5.46e-06 ***
lmktval      0.099872   0.049214   2.029  0.04397 *  
profmarg    -0.002211   0.002105  -1.050  0.29514    
ceoten       0.017104   0.005540   3.087  0.00236 ** 
comten      -0.009238   0.003337  -2.768  0.00626 ** 
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1

Residual standard error: 0.4947 on 171 degrees of freedom
Multiple R-squared:  0.3525,	Adjusted R-squared:  0.3336 
F-statistic: 18.62 on 5 and 171 DF,  p-value: 9.488e-15


In [25]:
reg2<-lm(lsalary ~ lsales + lmktval + profmarg + ceoten + comten + comtensq + ceotensq, data=ceosal2)
summary(reg2)

linearHypothesis(reg2, c("comtensq=ceotensq","ceotensq=0"))


Call:
lm(formula = lsalary ~ lsales + lmktval + profmarg + ceoten + 
    comten + comtensq + ceotensq, data = ceosal2)

Residuals:
     Min       1Q   Median       3Q      Max 
-2.47661 -0.26615 -0.03878  0.27787  1.89344 

Coefficients:
              Estimate Std. Error t value Pr(>|t|)    
(Intercept)  4.424e+00  2.656e-01  16.655  < 2e-16 ***
lsales       1.857e-01  3.980e-02   4.665 6.25e-06 ***
lmktval      1.018e-01  4.872e-02   2.089 0.038228 *  
profmarg    -2.575e-03  2.088e-03  -1.233 0.219137    
ceoten       4.772e-02  1.416e-02   3.371 0.000929 ***
comten      -6.063e-03  1.189e-02  -0.510 0.610815    
comtensq    -5.389e-05  2.528e-04  -0.213 0.831474    
ceotensq    -1.119e-03  4.814e-04  -2.324 0.021329 *  
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1

Residual standard error: 0.4892 on 169 degrees of freedom
Multiple R-squared:  0.3745,	Adjusted R-squared:  0.3486 
F-statistic: 14.45 on 7 and 169 DF,  p-value: 1.136e-14


,Res.Df,RSS,Df,Sum of Sq,F,Pr(>F)
,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
1,171,41.85600,NA,NA,NA,NA
2,169,40.43785,2,1.418157,2.963419,0.05433214


### RESET Test Explanation

**Null hypothesis (H₀):** your model is correctly specified.  

**Alternative hypothesis (H₁):** there’s misspecification (maybe omitted variables, wrong functional form, nonlinearity).  

---

**Steps:**

1. Estimate your original model (`reg1`). Get fitted values: $$\hat{y}$$.  
2. Create nonlinear terms of the fitted values: $$\hat{y}^2, \hat{y}^3, \dots$$  
3. Re-estimate the regression, adding those powers of fitted values as extra regressors:  

$$
lsalary = \beta_0 + \beta_1 lsales + \beta_2 lmktval + \beta_3 profmarg + \beta_4 ceoten + \beta_5 comten + \gamma_1 \hat{y}^2 + \gamma_2 \hat{y}^3 + u
$$

4. Test whether the added terms matter:  
   - **H₀:** $$\gamma_1 = \gamma_2 = 0$$ (no misspecification)  
   - **H₁:** at least one $$\gamma \neq 0$$  

---

**In practice:**
- First, create $$\hat{y}$$, then its square and cube.  
- Re-run the regression including them.  
- Use `linearHypothesis()` to test whether those coefficients are jointly zero.  

---

**Interpretation:**
- If the test is **not significant** (p-value > 0.05): keep the original model → no strong evidence of misspecification.  
- If the test is **significant** (p-value < 0.05): evidence of misspecification → the model may be missing nonlinear terms or other structure.  


In [ ]:
ceosal2$yhat <- reg1$fitted.values
ceosal2$yhatsq <- ceosal2$yhat^2
ceosal2$yhatcub <- ceosal2$yhat^3

reg_RESET <- lm(lsalary ~ lsales + lmktval + profmarg + ceoten + comten + yhatsq + yhatcub, data=ceosal2)
summary(reg_RESET)

linearHypothesis(reg_RESET, c("yhatsq=yhatcub","yhatcub=0"))


Call:
lm(formula = lsalary ~ lsales + lmktval + profmarg + ceoten + 
    comten + yhatsq + yhatcub, data = ceosal2)

Residuals:
     Min       1Q   Median       3Q      Max 
-2.48708 -0.24904 -0.02634  0.30058  1.94101 

Coefficients:
            Estimate Std. Error t value Pr(>|t|)  
(Intercept) 448.0913   185.4682   2.416   0.0168 *
lsales       36.0091    14.7567   2.440   0.0157 *
lmktval      19.2171     7.8588   2.445   0.0155 *
profmarg     -0.4228     0.1735  -2.437   0.0158 *
ceoten        3.2823     1.3445   2.441   0.0157 *
comten       -1.7736     0.7264  -2.441   0.0157 *
yhatsq      -28.2289    11.8159  -2.389   0.0180 *
yhatcub       1.3874     0.5909   2.348   0.0200 *
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1

Residual standard error: 0.4848 on 169 degrees of freedom
Multiple R-squared:  0.3856,	Adjusted R-squared:  0.3602 
F-statistic: 15.16 on 7 and 169 DF,  p-value: 2.661e-15


,Res.Df,RSS,Df,Sum of Sq,F,Pr(>F)
,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
1,171,41.85600,NA,NA,NA,NA
2,169,39.71571,2,2.14029,4.553728,0.01185193


some evidence of misspecification (p-value = 0.012 < 0.05).

In [28]:
gpa2 <- read_dta("data_ps2/GPA2.dta")

In [29]:
reg1<-lm(sat ~ hsize + hsizesq, data=gpa2)
summary(reg1)


Call:
lm(formula = sat ~ hsize + hsizesq, data = gpa2)

Residuals:
    Min      1Q  Median      3Q     Max 
-562.38  -93.07   -3.71   90.62  507.72 

Coefficients:
            Estimate Std. Error t value Pr(>|t|)    
(Intercept)  997.981      6.203 160.875  < 2e-16 ***
hsize         19.814      3.991   4.965 7.14e-07 ***
hsizesq       -2.131      0.549  -3.881 0.000106 ***
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1

Residual standard error: 138.9 on 4134 degrees of freedom
Multiple R-squared:  0.00765,	Adjusted R-squared:  0.007169 
F-statistic: 15.93 on 2 and 4134 DF,  p-value: 1.279e-07


yes, the quadratic term is significant to the 0% level.

we have that
$$
sat = \beta_0 + \beta1 hsize + \beta_2 hsize^2 + u 
$$
Thus, if we want the optimal value of `sat` according to the variable `hsize`, we need to compute the first derivative and equal it to zero:
$$
\frac{\text{d}sat}{\text{d}hsize} = \beta_1 + 2\beta_2 hsize = 0
$$
$$
\Leftrightarrow hsize = \frac{-\beta_1}{2\beta_2}
$$

In [33]:
opt_size <- coef(reg1)[2]/(-2*coef(reg1)[3])
opt_size

hsize 
4.649958

In [35]:
lsat <- log(gpa2$sat)
reg2<-lm(lsat ~ hsize + hsizesq, data=gpa2)
summary(reg2)


Call:
lm(formula = lsat ~ hsize + hsizesq, data = gpa2)

Residuals:
     Min       1Q   Median       3Q      Max 
-0.77744 -0.08493  0.00557  0.09465  0.40946 

Coefficients:
              Estimate Std. Error  t value Pr(>|t|)    
(Intercept)  6.8960291  0.0061515 1121.032  < 2e-16 ***
hsize        0.0196029  0.0039572    4.954 7.57e-07 ***
hsizesq     -0.0020872  0.0005444   -3.834 0.000128 ***
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1

Residual standard error: 0.1377 on 4134 degrees of freedom
Multiple R-squared:  0.007773,	Adjusted R-squared:  0.007293 
F-statistic: 16.19 on 2 and 4134 DF,  p-value: 9.885e-08


In [37]:
opt_size <- coef(reg2)[2]/(-2*coef(reg2)[3])
opt_size

hsize 
4.695923

In [38]:
gpa2$size<-gpa2$hsize*100
gpa2$sizesq<-gpa2$size^2

reg3<-lm(sat ~ size + sizesq, data=gpa2)
summary(reg3)

opt_size3<-coef(reg3)[2]/(-2*coef(reg3)[3])
opt_size3


Call:
lm(formula = sat ~ size + sizesq, data = gpa2)

Residuals:
    Min      1Q  Median      3Q     Max 
-562.38  -93.07   -3.71   90.62  507.72 

Coefficients:
              Estimate Std. Error t value Pr(>|t|)    
(Intercept)  9.980e+02  6.203e+00 160.875  < 2e-16 ***
size         1.981e-01  3.991e-02   4.965 7.14e-07 ***
sizesq      -2.131e-04  5.490e-05  -3.881 0.000106 ***
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1

Residual standard error: 138.9 on 4134 degrees of freedom
Multiple R-squared:  0.00765,	Adjusted R-squared:  0.007169 
F-statistic: 15.93 on 2 and 4134 DF,  p-value: 1.279e-07


size 
464.9958

The coefficients on the regressors are smaller in part (c): by factor 100 for size 
and by factor 10000 for sizesq. R-squared, F- and t-values are unchanged as expected. 
The optimal size is now size* = 0.1981/[2(0.000213)]  465 students, which is the 
same as in part (b). 